In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import audtorch.metrics.functional as audtorch

import time
from tqdm import tnrange, tqdm_notebook
import os
import matplotlib.pyplot as plt

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [2]:
class PartNonLinear(nn.Module):
    def __init__(self, unit_no, t_dim, k_dim, h_dim, p_dim, f_dim):
        super(PartNonLinear, self).__init__()
        self.unit_no = unit_no
        self.t_dim = t_dim
        self.h_dim = h_dim
        self.p_dim = p_dim
        self.k_dim = k_dim
        self.f_dim = f_dim
        
        self.featurize = nn.ModuleList([nn.Linear(self.t_dim,
                                                  self.f_dim) for i in range(self.unit_no)]).cuda()
        
        self.hidden1 = nn.ModuleList([nn.Linear(self.k_dim*self.f_dim,
                                               self.h_dim) for i in range(self.p_dim)]).cuda()
        self.hidden1_act = nn.ModuleList([nn.Tanh() for i in range(self.p_dim)]).cuda()
        
        self.output_layer = nn.ModuleList([nn.Linear(self.h_dim,
                                                    1) for i in range(self.p_dim)]).cuda()
        
    def forward(self, S, pix_units):
        
        F = torch.empty(S.shape[0], self.unit_no * self.f_dim).cuda()
        for n in range(self.unit_no):
            feat_n = self.featurize[n](S[:, n*self.t_dim : (n+1)*self.t_dim])
            F[:, n*self.f_dim : (n+1)*self.f_dim] = feat_n
        
        I = torch.empty(S.shape[0] , self.p_dim).cuda()
        
        for x in range(self.p_dim):
            unit_ids = pix_units[x]
            feat_ids = torch.empty((self.k_dim * self.f_dim))
            for i in range(self.k_dim):
                feat_ids[i*self.f_dim : (i+1)*self.f_dim] = torch.arange(self.f_dim) + unit_ids[i]*self.f_dim
            
            pix_feat = self.hidden1[x](F[:, feat_ids.long()])
            pix_feat = self.hidden1_act[x](pix_feat)

            out = self.output_layer[x](pix_feat)
            
            I[:, x] = out.reshape(-1)
            
        return I            

In [3]:
t_dim = 50
k_dim = 10
h_dim = 40
f_dim = 10

unit_no = 2094
p_dim = 95*146
#p_dim = 730

In [4]:
#test_I = np.load("/ssd/joon/2017_11_29_ns/images/hp_test_images.npy")
#test_I = np.load("/ssd/joon/2017_11_29_ns/images/i2000_p730_hp_test_images.npy")
test_I = np.load("/ssd/joon/2017_11_29_ns/images/smooth_test_images.npy")
test_I = torch.from_numpy(test_I)

#pixel_units = np.load("/ssd/joon/2017_11_29_ns/yass/l1_i2000_p730_pixel_units.npy")[:,:k_dim]
pixel_units = np.load("/ssd/joon/2017_11_29_ns/yass/yass_l1_pixel_units.npy")[:,:k_dim]
pixel_units = torch.from_numpy(pixel_units).cuda()
#pixel_units = torch.from_numpy(pixel_units)

#test_S = np.load("/ssd/joon/2017_11_29_ns/yass/neural/yass_i2000_50_test_neural.npy")
test_S = np.load("/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_test_neural.npy")
test_S = torch.from_numpy(test_S)

model = PartNonLinear(unit_no, t_dim, k_dim, h_dim, p_dim, f_dim)
model = model.float()
model.cuda()

PartNonLinear(
  (featurize): ModuleList(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): Linear(in_features=50, out_features=10, bias=True)
    (2): Linear(in_features=50, out_features=10, bias=True)
    (3): Linear(in_features=50, out_features=10, bias=True)
    (4): Linear(in_features=50, out_features=10, bias=True)
    (5): Linear(in_features=50, out_features=10, bias=True)
    (6): Linear(in_features=50, out_features=10, bias=True)
    (7): Linear(in_features=50, out_features=10, bias=True)
    (8): Linear(in_features=50, out_features=10, bias=True)
    (9): Linear(in_features=50, out_features=10, bias=True)
    (10): Linear(in_features=50, out_features=10, bias=True)
    (11): Linear(in_features=50, out_features=10, bias=True)
    (12): Linear(in_features=50, out_features=10, bias=True)
    (13): Linear(in_features=50, out_features=10, bias=True)
    (14): Linear(in_features=50, out_features=10, bias=True)
    (15): Linear(in_features=50, out_features=10, bia

In [5]:
ids = [2449, 2374, 2249, 2124, 1999, 1874, 1749, 1624, 1499, 1374, 1249, 1124, 999, 874, 749, 624, 499,
      374, 249, 124]

for i in tnrange(len(ids)):
    #index = (i+1)*125 - 1
    #index = ids[-i-1]
    index = ids[-i-1]
    
    print(index)
    
    time_1 = time.time()
    
    model.load_state_dict(torch.load("/ssd/joon/2017_11_29_ns/yass/low_pass/FULL_LP_f10_h40_0.001_w5e6_i"+str(index)+"_nn.pt"))
    
    time_2 = time.time()
    
    dec_I = model(test_S.float().cuda(), pixel_units.long())
    #dec_I = model(test_S.float(), pixel_units.long())
    
    time_3 = time.time()
    
    batch_corr = audtorch.pearsonr(test_I.T.cuda(), dec_I.T.cuda())
    mean_batch_corr = torch.mean(batch_corr)

    print(mean_batch_corr.item())

124
0.05745675511419295
249
0.21831357194818254
374
0.2070971364653983
499
0.1759417937712504
624
0.1585870559492541
749


KeyboardInterrupt: 

In [5]:
model.load_state_dict(torch.load("/ssd/joon/2017_11_29_ns/yass/high_pass/FULL_f10_h40_0.5_w5e6_i2249_nn.pt"))
model.cuda()
dec_I = model(test_S.float().cuda(), pixel_units.long().cuda())
dec_I = dec_I.cpu().detach().numpy()
print(dec_I.shape)
print(type(dec_I))
print(dec_I.dtype)
np.save("/ssd/joon/2017_11_29_ns/yass/decoded/hp_FULL_f10_h40_0.5_w5e6.npy", dec_I)

(100, 13870)
<class 'numpy.ndarray'>
float32
